# ¿Cuáles son las principales características que llevan a que se ROBEN UNA MOTO O UN VEHÍCULO? 
## Seguro está en los datos

### Data processing step

Let's load all the dataset:

In [11]:
import numpy as np
import pandas as pd
import requests
import json

url = 'https://cali.ckan.io/api/3/action/datastore_search?resource_id=f465aa6b-a49f-42ce-8da2-fef524e7eada&q={"municipio":"cali"}'
JSONContent = requests.get(url).json()

Finding the column names and the records:

In [12]:
fields = JSONContent['result']['fields']
features = list(map(lambda field: field['id'], fields))
print('Column names: \n', features)

Column names: 
 ['_id', 'fecha', 'departamento', 'municipio', 'dia_semana', 'hora', 'barrio', 'zona', 'clase_sitio', 'arma_empleada', 'movil_agresor', 'movil_victima', 'edad', 'sexo', 'estado_civil', 'pais_nacimiento', 'clase_empleado', 'profesion', 'escolaridad', 'codigo_dane', 'cantidad', 'anio', 'mes', 'utc_datetime', 'dept_code_dane', 'muni_code_dane', 'code_dane']


In [13]:
def getDataframe(array, columnNames):
    df = pd.DataFrame()
    for column in columnNames:
        df[column]=list(map(lambda record: record[column], array))
    return df

In [14]:
records = JSONContent['result']['records']
df = getDataframe(records, features)
tamanio = len(df)
df.head()

,_id,fecha,departamento,municipio,dia_semana,hora,barrio,zona,clase_sitio,arma_empleada,...,profesion,escolaridad,codigo_dane,cantidad,anio,mes,utc_datetime,dept_code_dane,muni_code_dane,code_dane
0,640,2019-07-01,VALLE DEL CAUCA,CALI,Lunes,00:01:00,LA RIVERA I E6,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,...,NO REPORTADA,TECNICO,76001000,1,2019,Julio,2019-07-01T00:01:00Z,76,76001,76-76001
1,604,2019-07-01,VALLE DEL CAUCA,CALI,Lunes,19:00:00,SAN PASCUAL E3,URBANA,ESTACION M.I.O.,SIN EMPLEO DE ARMAS,...,ARQUITECTURA,SUPERIOR,76001000,1,2019,Julio,2019-07-01T19:00:00Z,76,76001,76-76001
2,605,2019-07-01,VALLE DEL CAUCA,CALI,Lunes,10:00:00,REPUBLICA ISRAEL E16,URBANA,VIAS PUBLICAS,ARMA BLANCA / CORTOPUNZANTE,...,LICENCIADO,SUPERIOR,76001000,1,2019,Julio,2019-07-01T10:00:00Z,76,76001,76-76001
3,606,2019-07-01,VALLE DEL CAUCA,CALI,Lunes,20:00:00,GRANADA E2,URBANA,VIAS PUBLICAS,ARMA BLANCA / CORTOPUNZANTE,...,AUXILIARES DE ALMACEN Y BODEGA,SECUNDARIA,76001000,1,2019,Julio,2019-07-01T20:00:00Z,76,76001,76-76001
4,607,2019-07-01,VALLE DEL CAUCA,CALI,Lunes,18:30:00,EL LIDO E19,URBANA,VIAS PUBLICAS,SIN EMPLEO DE ARMAS,...,PSICOLOGIA,SUPERIOR,76001000,1,2019,Julio,2019-07-01T18:30:00Z,76,76001,76-76001


Due to there are only 100 records, let's proceed to iterate to find the other ones

In [20]:
baseURL='https://cali.ckan.io'
links=JSONContent['result']['_links']
while(tamanio > 0):
    nextURL=JSONContent['result']['_links']['next']
    url = f"{baseURL}{nextURL}"
    JSONContent = requests.get(url).json()
    records = JSONContent['result']['records']
    new_df = getDataframe(records, features)
    tamanio = len(new_df)
    print('TAMANIO:', len(df))
    df = pd.concat([df, new_df])

TAMANIO: 96300
TAMANIO: 96353


In [21]:
#df_big = df.copy()
len(df)

96353

### Data exploration

Let's explore de data:

In [22]:
fields = JSONContent['result']['fields']
df_info =  pd.DataFrame()
df_info['type'] = list(map(lambda field: field['type'], fields))
df_info['id'] = list(map(lambda field: field['id'], fields))
df_info['label'] = list(map(lambda field: field['info']['label'] if 'info' in field else '', fields))
df_info['note'] = list(map(lambda field: field['info']['notes'] if 'info' in field else '', fields))
df_info

,type,id,label,note
0,int,_id,,
1,text,fecha,fecha,Fecha en la cual se registra el delito [año-me...
2,text,departamento,departamento,Departamento en el cual ocurrió el delito
3,text,municipio,municipio,Municipio en el cual ocurrió el delito
4,text,dia_semana,dia_semana,Día de la semana en la cual ocurrió el delito ...
5,text,hora,hora,Hora en la cual ocurrió el delito [00-24]
6,text,barrio,barrio,Barrio en el cual ocurrió el delito
7,text,zona,zona,"Zona en el cual ocurrió el delito [Rural, Urbana]"
8,text,clase_sitio,clase_sitio,Clase de sitio donde ocurrió el delito
9,text,arma_empleada,arma_empleada,Arma empleada para cometer el delito


In [23]:
df.describe(include=np.object)

,fecha,departamento,municipio,dia_semana,hora,barrio,zona,clase_sitio,arma_empleada,movil_agresor,...,profesion,escolaridad,codigo_dane,cantidad,anio,mes,utc_datetime,dept_code_dane,muni_code_dane,code_dane
count,96353,96353,96353,96353,96353,96353,96353,96353,96353,96353,...,96353,96353,96353,96353,96353,96353,96353,96353,96353,96353
unique,3410,1,1,7,1256,424,3,212,11,14,...,1001,8,1,4,10,12,76695,1,1,1
top,2019-12-30,VALLE DEL CAUCA,CALI,Viernes,00:00:00,SAN PEDRO E3,URBANA,VIAS PUBLICAS,SIN EMPLEO DE ARMAS,A PIE,...,-,SECUNDARIA,76001000,1,2017,Octubre,2017-10-31T00:00:00Z,76,76001,76-76001
freq,126,96353,96353,15829,3571,3324,95109,61771,37140,67584,...,72307,54986,96353,96132,13786,9821,18,96353,96353,96353


In [24]:
df.to_csv("HURTO_PERSONAS.csv")